In [1]:
import pandas as pd
import re
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#tokenizerSumLegal = AutoTokenizer.from_pretrained("nsi319/legal-led-base-16384")  
#modelSumLegal = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-led-base-16384")
#summarizerFacebook = pipeline("summarization", model="facebook/bart-large-cnn")
#tokenizerFaceBook = summarizerFacebook.tokenizer

################### LAWMARKCASES ##################################
FILE_LAWMARK = '/home/jaco/Projetos/landMarkClassification/data/scraper/scrapee.csv'

dfLawMark = pd.read_csv(FILE_LAWMARK)

dfLawMark['text'] = dfLawMark['text'].astype(str)
dfLawMark['subject'] = dfLawMark['subject'].astype(str)

#tirar numeros? fazer uma limpeza? Tirar os 300 nomes de advogados q tem? Tirar os nomes das leis?
#tirar os numeros q marcam os paragrafos? Syllabus e Opinion ja resolvem?

def limpeza_basica(texto):

    texto = texto['text']

    texto = texto[169:-124] ### tirando uns html que nao sairam

    texto = re.sub('\. [0-9]+ ', ' ', texto)
    texto = re.sub('\* \* \*', ' ', texto)
    texto = re.sub('\.\' [0-9]+ ', ' ', texto)
    texto = re.sub('\.+ \.+ \.+', ' ', texto)
    texto = re.sub('\[.+?omitted\]', '', texto)
    texto = re.sub('"\." [0-9]+ ', ' ', texto)
    texto = re.sub('\' [0-9]+ [a-z]', ' ', texto,flags=re.IGNORECASE) #deixar todos com essa parada de ignore case?

    texto = re.sub('"\." [0-9]+ ', ' ', texto)

    texto = re.sub('\.[0-9]+ ', '.', texto) #pode mexer muito, muito especifico
    texto = re.sub('\. [0-9]+ ', ' ', texto) #pode mexer muito, muito especifico

    texto = re.sub(' +',' ',texto)

    return texto

dfLawMark['text'] = dfLawMark.apply(lambda row : limpeza_basica(row), axis=1)

#dfLawMark.to_csv('/home/jaco/Projetos/landMarkClassification/data/checkAplicacaoRegex.csv',index=False)

dfLawMark = dfLawMark[['text','subject']]

dfLawMark = dfLawMark[(dfLawMark['text'] != '')] ### checar depois pq aparece isso, o scrape pode retornar nulo? 
dfLawMark = dfLawMark[dfLawMark["text"].apply(lambda x: len(x) > 1000)] ### checar depois pq aparece isso, pq tem processos que vem do scraper que nao puxa quase nada?
dfLawMark = dfLawMark.sample(frac=1)

print(f'tamanho do dataset: {len(dfLawMark)}')

##testando classificações mais facieis:
## tirando classes sem samples suficientes:
min_sample_size = 14
df2 = dfLawMark.groupby(['subject'],as_index=False).count()
df2 = df2[df2['text'] >= min_sample_size]
lista = list(df2['subject'])

#####
#dfLawMark = dfLawMark[dfLawMark['subject'].isin(lista)]
dfLawMark = dfLawMark[dfLawMark['subject'].isin(['First Amendment','Freedom of Speech','Criminal Law','Racial Discrimination'])]
#####

DATSET_SIZE = len(dfLawMark)
ROW_COUNTER = 0

print(f'tamanho do dataset resumido: {len(dfLawMark)}')

### checar se o webscrape ta perfeito vs o site.
### pegar um sumarizador profissional law:

/home/jaco/.pyenv/versions/3.9.7/envs/landMarkClassification/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tamanho do dataset: 2271
tamanho do dataset resumido: 393


In [2]:
dfLawMark.head(5)

,text,subject
1149,"431 U.S 97 S.Ct 52 L.Ed.2d 675 Hugh CAREY, etc...",Freedom of Speech
237,"135 U.S 10 S.Ct 34 L.Ed CUNNINGHAM, Sheriff,v....",Criminal Law
262,317 U.S 63 S.Ct 87 L.Ed 87 L.Ed Ex parte QUIRI...,Criminal Law
1817,109 U.S 3 S.Ct 27 L.Ed 'THE CIVIL RIGHTS CASES...,Racial Discrimination
1834,"349 U.S 75 S.Ct 99 L.Ed Oliver BROWN, et al., ...",Racial Discrimination


In [3]:
def split_text(text,char_len):

    text = text['text']
    
    arr = []
    tmp_arr = []
    char_counter = 0

    for n in text:

        char_counter = char_counter + 1

        tmp_arr.append(n)

        if char_counter >= char_len:
            arr.append([''.join(tmp_arr)])
            char_counter = 0
            tmp_arr = []

    return arr

def ajuste_imbecil(text):
    text = text['split']
    return text[0]

char_num = 2000

dfLawMark['split'] = dfLawMark.apply(lambda row : split_text(row,char_num), axis=1)
dfLawMark = dfLawMark.explode('split')
dfLawMark['split'] = dfLawMark.apply(lambda row : ajuste_imbecil(row), axis=1)
dfLawMark = dfLawMark[dfLawMark["text"].apply(lambda x: len(x) > char_num*0.95)]
dfLawMark['Syllabus'] = dfLawMark['split']
dfLawMark.drop(['split'],inplace=True, axis=1)

In [5]:
encodedClass = pd.factorize(dfLawMark['subject'])
dfLawMark['encoded'] = encodedClass[0]+1 ## pytorch precisa começar do 1

dfEnCode = dfLawMark[['subject','encoded']]
dfEnCode.drop_duplicates(inplace=True)

dfEnCode.to_csv('/home/jaco/Projetos/landMarkClassification/data/enCode_Split.csv',index=False)
dfLawMark.to_csv('/home/jaco/Projetos/landMarkClassification/data/onlyLandMarkWSyllabus_Split.csv',index=False)

/tmp/ipykernel_28139/2613381184.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfEnCode.drop_duplicates(inplace=True)


In [10]:
len(dfLawMark)

15458

In [9]:
from sklearn.model_selection import train_test_split

X = dfLawMark['Syllabus']
y = dfLawMark['encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=666, stratify=y)

dfLawMark_Train = pd.DataFrame()
dfLawMark_Train['Syllabus'] = X_train.to_frame()
dfLawMark_Train['encoded'] = y_train.to_frame()

dfLawMark_Test = pd.DataFrame()
dfLawMark_Test['Syllabus'] = X_test.to_frame()
dfLawMark_Test['encoded'] = y_test.to_frame()

dfLawMark_Train.to_csv('/home/jaco/Projetos/landMarkClassification/data/onlyLandMarkWSyllabus_Train_Split.csv',index=False)
print(f'tamanho do train dataset: {len(dfLawMark_Train)}')
dfLawMark_Test.to_csv('/home/jaco/Projetos/landMarkClassification/data/onlyLandMarkWSyllabus_Test_Split.csv',index=False)
print(f'tamanho do test dataset: {len(dfLawMark_Test)}')

tamanho do train dataset: 12366
tamanho do test dataset: 3092


In [ ]:
mediumBBC = pd.read_csv('/home/jaco/Projetos/landMarkClassification/data/bbc-text.csv')
mediumBBC.rename(columns={'category':'subject'},inplace=True)
mediumBBC.rename(columns={'text':'Syllabus'},inplace=True)

encodedClass = pd.factorize(dfLawMark['subject'])
dfLawMark['encoded'] = encodedClass[0]+1 ## pytorch precisa começar do 1

dfEnCode = dfLawMark[['subject','encoded']]
dfEnCode.drop_duplicates(inplace=True)

dfEnCode.to_csv('/home/jaco/Projetos/landMarkClassification/data/enCode_Split.csv',index=False)

In [ ]:
X = mediumBBC['Syllabus']
y = mediumBBC['encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=666, stratify=y)

mediumBBC_Train = pd.DataFrame()
mediumBBC_Train['Syllabus'] = X_train.to_frame()
mediumBBC_Train['encoded'] = y_train.to_frame()

mediumBBC_Test = pd.DataFrame()
mediumBBC_Test['Syllabus'] = X_test.to_frame()
mediumBBC_Test['encoded'] = y_test.to_frame()

mediumBBC_Train.to_csv('/home/jaco/Projetos/landMarkClassification/data/mediumBBC_Train.csv',index=False)
print(f'tamanho do train dataset: {len(mediumBBC_Train)}')
mediumBBC_Test.to_csv('/home/jaco/Projetos/landMarkClassification/data/mediumBBC_Test.csv',index=False)
print(f'tamanho do test dataset: {len(mediumBBC_Test)}')